# CSS 444 Final Project

### Mandy Chen, Mustafa Sabir, Mychael Johnson, Michael Cho

# Baltimore, MD Arrest Records 12/31/2009 - 10/22/2021

This dataset is all arrest records in Baltimore, MD from 12/31/2009 to 10/22/2021. We found this dataset on the [Police Data Initiative](https://www.policedatainitiative.org/datasets/) database website, which contains various types of datasets from police jurisdictions across the United States. 

[Open Baltimore Arrest Dataset](https://data.baltimorecity.gov/datasets/baltimore::arrests/about)

In [1]:
# Importing required Python Libraries

import pandas as pd
from matplotlib import pylab
import numpy as np
import sklearn
from sklearn import linear_model
import sklearn.preprocessing as preprocessing
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from statsmodels.stats import proportion
import datetime
from itertools import product
import csv

# Code to display whole dataframe with print
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
# Data set file path
data_url = '/Users/michaelcho/_GoogleDrive/_Classes/CSS 444 A/Final_Project/data/Arrests.csv'

In [3]:
# Loading dataset into pd DataFrame
data = pd.read_csv(data_url)

# Descriptive Analysis

In [4]:
train_data = data.sample(frac=0.8, random_state=25)
test_data = data.drop(train_data.index)

num_train = len(train_data)

print(f"No. of training points: {train_data.shape[0]}")
print(f"No. of testing points: {test_data.shape[0]}")

original = pd.concat([train_data, test_data])
original = original[original['Gender'].notna()]

original.head()

No. of training points: 281255
No. of testing points: 70314


,X,Y,RowID,ArrestNumber,Age,Gender,Race,ArrestDateTime,ArrestLocation,IncidentOffence,IncidentLocation,Charge,ChargeDescription,District,Post,Neighborhood,Latitude,Longitude,GeoLocation,Shape
171271,1.423933e+06,591809.446923,171272,12505619.0,55.0,M,B,2012/06/29 11:15:00+00,800 FAYETTE ST,87NARCOTICS,800 FAYETTE ST,4 3550,CDS VIOL,Southeast,211,Jonestown,39.2910,-76.6057,"(39.291,-76.6057)",NaN
346948,1.442387e+06,586213.654760,346949,10000971.0,24.0,M,B,2010/01/06 09:15:00+00,1500 CHARLOTTE AV,3KROBB RES. (UA),1500 CHARLOTTE AV,NaN,Unknown Charge,Southeast,234,Broening Manor,39.2754,-76.5406,"(39.2754,-76.5406)",NaN
10906,1.416643e+06,581908.237423,10907,15188371.0,26.0,M,B,2015/12/16 10:30:00+00,2200 CEDLEY ST,87NARCOTICS,2200 CEDLEY ST,4 3550,CDS,Southern,921,Westport,39.2639,-76.6316,"(39.2639,-76.6316)",NaN
218363,NaN,NaN,218364,14076816.0,23.0,F,W,2014/05/12 15:00:00+00,NaN,Unknown Offense,NaN,1 0088,VIOLATION OF PROBATION,NaN,NaN,NaN,NaN,NaN,"(,)",NaN
341962,NaN,NaN,341963,22003143.0,30.0,M,B,2022/01/10 07:00:00+00,NaN,Unknown Offense,NaN,1 0077,FAILURE TO APPEAR,NaN,NaN,NaN,NaN,NaN,"(,)",NaN


In [5]:
print("Total Female: " + str(original['Gender'].value_counts().F))
print("Total Male: " + str(original['Gender'].value_counts().M))

print("Percentage Female: " + str((original['Gender'].value_counts().F/len(original))))
print("Percentage Male: " + str((original['Gender'].value_counts().M/len(original))))

Total Female: 65151
Total Male: 286379
Percentage Female: 0.18533553324040622
Percentage Male: 0.8146644667595938


In [6]:
black = original[original['Race']=='B'] #subgrouping
white = original[original['Race']=='W']
total_black = len(original[original['Race']=='B'])
total_white = len(original[original['Race']=='W'])

print(total_black)
print(total_white)


black_female = len(black[(black.Gender == 'F') | (black.Gender == 'F')])/total_black
black_male = len(black[(black.Gender == 'M') | (black.Gender == 'M')])/total_black

white_female = len(white[(white.Gender == 'F') | (white.Gender == 'F')])/total_white
white_male = len(white[(white.Gender == 'M') | (white.Gender == 'M')])/total_white

target_percent = pd.DataFrame({'F': [black_female, white_female], 'M': [black_male, white_male]}, index=['Black', 'White'])
target_percent

290485
52200


,F,M
Black,0.162604,0.837396
White,0.320613,0.679387


# Logistic Regression

In [7]:
# Creating a Binary Target Variable
# In this case, whether the arrested person is male
is_male = original['Gender']
is_male = is_male.replace('M', 1).replace('F', 0)

In [8]:
# Using race as the only feature
data_race = original['Race']
data_race.head()

171271    B
346948    B
10906     B
218363    W
341962    B
Name: Race, dtype: object

In [9]:
# Function for normalizing data
def data_transform(df):
    """Normalize features."""
    binary_data = pd.get_dummies(df)
    feature_cols = binary_data[binary_data.columns[:]]
    scaler = preprocessing.StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(feature_cols), columns=feature_cols.columns)
    return data

#see here for transformation : https://developers.google.com/machine-learning/data-prep/transform/normalization

In [10]:
# Data normalized
data = data_transform(data_race)

# Labels changed
data = data.rename(columns={"A": "Asian", "B": "Black", "H": "Hispanic", "I": "NativeAmerican", "U": "Unknown", "W": "White"})
data.head()

,Asian,Black,Hispanic,NativeAmerican,Unknown,White
0,-0.051614,0.458420,-0.001687,-0.048531,-0.14341,-0.417600
1,-0.051614,0.458420,-0.001687,-0.048531,-0.14341,-0.417600
2,-0.051614,0.458420,-0.001687,-0.048531,-0.14341,-0.417600
3,-0.051614,-2.181408,-0.001687,-0.048531,-0.14341,2.394638
4,-0.051614,0.458420,-0.001687,-0.048531,-0.14341,-0.417600


In [12]:
# Logistic Regression Functions
cls_male = linear_model.LogisticRegression()
cls_male.fit(data,is_male)

# Printing coefficients for race
print(cls_male.coef_)

[[ 0.00682599  0.1366492   0.00974089  0.02142998  0.09607215 -0.18749153]]


In [14]:
from sklearn.metrics import mean_squared_error
res_male=cls_male.predict(data)

# Standard error measurement
print(mean_squared_error(is_male,res_male))

0.18533553324040622


# Independence

# Seperation

### Data Analysis

In [ ]:
original.info()

In [ ]:
original.isnull().sum()

In [ ]:
original.describe()

In [ ]:
feature = {
            "Row ID" : original['RowID'],
            "Age" : original['Age'], 
            "Gender" : original['Gender'],
            "Race" : original['Race'],
            "Arrest_Time" : original['ArrestDateTime'],
            "Incident_Offence" : original['IncidentOffence'],
            "Charge_Description" : original['ChargeDescription']
          }
marginal = pd.DataFrame(data=feature, columns=['Row ID', 'Age', 'Gender', 'Race', 'Arrest_Time', 'Incident_Offence', 'Charge_Description'])
marginal.head()

In [ ]:
#Prints information of all columns:
print("Marginal Data information:")
marginal.info()
# Number of Nan data
print("\n\nMarginal Data Size: ", len(marginal))
print("\n\nNumber of nan data: ")
marginal.isnull().sum()

In [ ]:
#Drop rows with Nan value
marginal.dropna(subset = ['Age'], inplace=True)

In [ ]:
print("Marginal data size after drop rows with Nan value: ", len(marginal))
print("\nNumber of nan data: ")
# Number of Nan data
marginal.isnull().sum()

In [ ]:
marginal.head()

In [ ]:
marginal.describe()

In [ ]:
# Subgroup with age 15–19, 20–29, 30–39, 40–49, and 50+
age_group = marginal['Age']

teen = marginal[age_group.between(15,19)]
twenties = marginal[age_group.between(20,29)]
thirties = marginal[age_group.between(30,39)]
forties =marginal[age_group.between(40,49)]
fifties_more = marginal[age_group >= 50]

total_teen = len(teen)
total_twenties = len(twenties)
total_thirties = len(thirties)
total_forties = len(forties)
total_fifties_more = len(fifties_more)

age_title = '15–19', '20–29', '30–39', '40–49', '50+'
age_val = [total_teen, total_twenties, total_thirties, total_forties, total_fifties_more]
colors = ['r','c','b','g','y']

# Create a pieplot
plt.pie(age_val, labels=age_title, colors=colors,
        wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' },
        startangle=0,
        explode = (0, 0, 0, 0, 0),
        autopct = '%1.2f%%')

plt.axis('equal') # Try commenting this out.
plt.title('Arrested Age Group')
plt.show();

In [ ]:
# get the count of each race in the race category
marginal.groupby('Race').count()

In [ ]:
# Race Pie Chart
race_title = 'Asian', 'Black', 'Hispanic', 'Native American', 'White', 'Unknown'
race_val = [931, 290403, 1, 826, 7079, 52193]
colors = ['c','y', 'k','r','g','m']

# Create a pieplot
plt.pie(race_val, labels=race_title, labeldistance=1.15, 
        wedgeprops = { 'linewidth' : 0.4, 'edgecolor' : 'white' },
        colors=colors,
        startangle=90,
        explode = (0, 0, 0.2, 0.4, 0.6, 0),
        autopct = '%1.2f%%')

plt.axis('equal')
plt.title('Arrested Race Group\n')

plt.show();

In [ ]:
# Subgroup Gender
gender_group = marginal['Gender']
female = marginal[gender_group == 'F']
male = marginal[gender_group =='M']

# Gender Pie Chart
gender_title = 'Male', 'Female'
gender_val = [len(male), len(female)]
colors = ['b','m']

plt.pie(gender_val, labels=gender_title, labeldistance=1.15,
        wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'white' },
        colors=colors,
        startangle=90,
        explode = (0, 0),
        autopct = '%1.2f%%')

plt.axis('equal')
plt.title('Arrested Gender Group\n')

plt.show();

### ROC Curve

In [ ]:
# target
is_male = marginal['Gender']
is_male = is_male.replace('M', 1).replace('F', 0)

# drop redundent column
del marginal["Gender"]
marginal.head()

In [ ]:
# normalize the data
data = marginal
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in data.columns:
  if data[column_name].dtype == object:
    data[column_name] = le.fit_transform(data[column_name])
  else:
    pass
data.head()

In [ ]:
# split data to with train data and test data with ratio 0.7:0.3
from sklearn.model_selection import train_test_split
train_data, test_data, train_label, test_label = train_test_split(data, is_male, test_size = 0.3)
train_num = len(train_data)

In [ ]:
log_re = LogisticRegression();
log_re.fit(train_data, train_label)
# keep probabilities for the positive outcome only
score = log_re.predict_proba(test_data)[:,1] 
log_re.coef_

In [ ]:
# Redo the marginal data frame since the original marginal is overwrite with the normalized data
marginal = pd.DataFrame(data=feature, columns=['Row ID', 'Age', 'Gender', 'Race', 'Arrest_Time', 'Incident_Offence', 'Charge_Description'])
# drops rows with Nan value based on the Age column
marginal.dropna(subset = ['Age'], inplace=True)

In [ ]:
original_test = marginal[train_num:]
dataFrame = {'target': test_label.values,
             'score':  score,           
             'race': original_test['Race']
             }
marginal2 = pd.DataFrame(data=dataFrame, columns=['target', 'score', 'race'])
marginal2.head() 

In [ ]:
# ploting ROC Curves
white = marginal2[marginal2['race'] == 'W']
fpr_white, tpr_white, _ = metrics.roc_curve(white['target'], white['score'])
auc_white = auc(fpr_white,tpr_white)

black = marginal2[marginal2['race'] == 'B']
fpr_black, tpr_black, _ = metrics.roc_curve(black['target'], black['score'])
auc_black = auc(fpr_black,tpr_black)

nativeAm = marginal2[marginal2['race'] == 'I']
fpr_na, tpr_na, _ = metrics.roc_curve(nativeAm['target'], nativeAm['score'])
auc_na = auc(fpr_na,tpr_na)

asian = marginal2[marginal2['race'] == 'A']
fpr_asian, tpr_asian, _ = metrics.roc_curve(asian['target'], asian['score'])
auc_asian = auc(fpr_asian,tpr_asian)


plt.figure(1, figsize=(6, 5))
plt.title('ROC curves by Race')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.plot(fpr_white, tpr_white, label='white (AUC = %0.3f)' % auc_white, linestyle='-', color='g')
plt.plot(fpr_black, tpr_black, label='black (AUC = %0.3f)' % auc_black, linestyle=':', color='r')
plt.plot(fpr_na, tpr_na, label='Native American (AUC = %0.3f)' % auc_na, linestyle='--', color='m')
plt.plot(fpr_asian, tpr_asian, label='Asian (AUC = %0.3f)' % auc_asian, linestyle='-.', color='b')


plt.legend()
plt.show()

# Relaxation of Seperation

# Equal Opportunity